<a href="https://colab.research.google.com/github/shivanishimpi/ML_PyTorch/blob/master/PyTorch_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Imports
import numpy as np
import torch

```PyTorch``` is for processing tensors (Numbers, Vectors, 3 Dimensional arrays)

Reference Links:


[For more information on Tensors](https://pytorch.org/docs/stable/tensors.html)

[PyTorch AutoGrad](https://towardsdatascience.com/pytorch-autograd-understanding-the-heart-of-pytorchs-magic-2686cd94ec95)

#Table of Contents

[Basics of PyTorch](#scrollTo=0eg8vAOQzChK)

[Linear Regression from Scratch](#scrollTo=FOtz5Oy3F0H9)

[Linear Regression using PyTorch built-ins](#scrollTo=eFPRLkl_-V0J)

#Basics of PyTorch

Working with Torch tensors 


In [ ]:
#number
tensor1 = torch.tensor(4.)
tensor1.dtype
#vector
tensor1 = torch.tensor([1,3.5,2,4,3.])
tensor1.dtype
#matrix
tensor1 = torch.tensor([[1,3],[2,4.],[2,4]])
tensor1
#3darray
tensor1 = torch.tensor(
    [
     [[1,2],[4,5]],
     [[2,3],[23,34]]
    ]
)
tensor1.shape

torch.Size([2, 2, 2])

Operations and gradients in Tensors

In [ ]:
'''
m = torch.tensor([2,3,4])
x = torch.tensor([42,35,62])
c = torch.tensor([4,5,6.], requires_grad=True) #only floats need gradients
y=m*x+c 
print(f'Operation: {y}')
'''

"\nm = torch.tensor([2,3,4])\nx = torch.tensor([42,35,62])\nc = torch.tensor([4,5,6.], requires_grad=True) #only floats need gradients\ny=m*x+c \nprint(f'Operation: {y}')\n"

In [ ]:
m = torch.tensor(6. ,requires_grad=True)
x = torch.tensor(2., requires_grad=True)
c = torch.tensor(4., requires_grad=True) #only floats need gradients
y=m*x+c
y.backward() #works only for scalars, computes gradients

print(f'''Displaying gradients / derivatives:
y = mx + c = {y}
dy/dx = {x.grad}
dy/dm = {m.grad}
dy/dc = {c.grad}
''')

Displaying gradients / derivatives:
y = mx + c = 16.0
dy/dx = 6.0
dy/dm = 2.0
dy/dc = 1.0



Interoperability with ```Numpy```

In [ ]:
import numpy as np

print(f'Input Torch Tensor: {y.dtype}')

#from torch tensor to numpy array
x = y.detach().numpy() #for gradients, else y.numpy()
print(f'Converted to Numpy Array: {x.dtype}')

#from numpy array to torch tensor
y = torch.from_numpy(x)
print(f'Reconverted to Torch Tensor: {y.dtype}')

Input Torch Tensor: torch.float32
Converted to Numpy Array: float32
Reconverted to Torch Tensor: torch.float32


#Linear Regression from the Scratch

We aren't using any input files here, since this is just s test implementation of model so we are going to create our own data

*Data creation*

In [ ]:
#Not gonna work since np.random.uniform doesn't take the dtype argument and it'd naturally be in float64
'''
inputs = np.random.uniform(low=20., high=99., size=(5,3))
labels = np.random.uniform(low=50., high=150., size=(5,2))
print(f'Inputs: {inputs}\nLabels: {labels}')
'''

"\ninputs = np.random.uniform(low=20., high=99., size=(5,3))\nlabels = np.random.uniform(low=50., high=150., size=(5,2))\nprint(f'Inputs: {inputs}\nLabels: {labels}')\n"

In [ ]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70]], dtype='float32')
# Targets (apples, oranges)
labels = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119]], dtype='float32')

In [ ]:
inputs = torch.from_numpy(inputs)
labels = torch.from_numpy(labels)

In [ ]:
inputs.dtype #if dtype!='float32' print(runtimeError)

torch.float32

Creating the linear regresssion model 


The equations are going to be - 



$$labels^{1} = w_{11}\cdot inputs^{1} + w_{12}\cdot inputs^{2}+ w_{13}\cdot inputs^{3}+b_{1}$$

$$labels^{2} = w_{21}\cdot inputs^{1} + w_{22}\cdot inputs^{2}+ w_{23}\cdot inputs^{3}+b_{2}$$

In [ ]:
weights = torch.randn(2,3,requires_grad=True) #2x3 weights matrix
bias = torch.randn(2,requires_grad=True) #two bias units
print(f'''
{weights}
{bias}
''')


tensor([[ 0.7391, -0.7679, -1.5919],
        [ 0.3780, -0.6600, -0.2270]], requires_grad=True)
tensor([-0.2100, -0.4485], requires_grad=True)



In [ ]:
def model(x):
  return x@weights.t() + bias #@ is matrix multiplication and .t() is transpose

In [ ]:
predictions = model(inputs)
print(predictions)
print(labels)

tensor([[ -66.1574,  -26.8358],
        [-102.4098,  -38.6584],
        [-131.1404,  -69.1672],
        [ -16.7404,    1.3260],
        [-134.3658,  -53.6150]], grad_fn=<AddBackward0>)
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


Mean Squared Error (MSE) Loss

In [ ]:
#delta = (predictions - labels)**2
#print(torch.sum(delta)/delta.numel())

In [ ]:
def MSE(val1,val2):
  delta = (val1-val2)**2
  return torch.sum(delta)/delta.numel()

In [ ]:
loss = MSE(predictions, labels)
print(loss)

tensor(26979.6934, grad_fn=<DivBackward0>)


In [ ]:
loss.backward(create_graph=True)

PyTorch accumulates gradients, so if ```.backward()``` is called again, it'll add the new gradients to the former one, rather than replacing it

In [ ]:
weights.grad.zero_()
bias.grad.zero_()

tensor([0., 0.], grad_fn=<ZeroBackward>)

Gradient descent

```torch.no_grad()``` avoids any gradient modification while updating the weights and biases

In [ ]:
numEpoch = 100
learning_rate = 1e-4

for i in range(numEpoch):
  predictions = model(inputs)
  loss = MSE(predictions,labels)
  loss.backward()
  with torch.no_grad():
    weights -= weights.grad * learning_rate
    bias -= bias.grad * learning_rate
    weights.grad.zero_()
    bias.grad.zero_()

In [ ]:
predictions = model(inputs)
loss = MSE(predictions, labels)
print(loss)

tensor(25.3096, grad_fn=<DivBackward0>)


In [ ]:
print(predictions)
print(labels)

tensor([[ 57.7079,  70.4623],
        [ 77.5975,  98.9614],
        [128.3031, 136.5970],
        [ 23.9365,  38.1593],
        [ 92.1946, 115.4091]], grad_fn=<AddBackward0>)
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


#Linear Regression Using PyTorch built-ins

https://youtu.be/vo_fUOk-IKk?list=PLWKjhJtqVAbm3T2Eq1_KgloC7ogdXxdRa&t=5567

In [ ]:
import torch.nn as nn

In [ ]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], [91, 88, 64], [87, 134, 58], 
                   [102, 43, 37], [69, 96, 70], [73, 67, 43], 
                   [91, 88, 64], [87, 134, 58], [102, 43, 37], 
                   [69, 96, 70], [73, 67, 43], [91, 88, 64], 
                   [87, 134, 58], [102, 43, 37], [69, 96, 70]], 
                  dtype='float32')

# Targets 
labels = np.array([[56, 70], [81, 101], [119, 133], 
                    [22, 37], [103, 119], [56, 70], 
                    [81, 101], [119, 133], [22, 37], 
                    [103, 119], [56, 70], [81, 101], 
                    [119, 133], [22, 37], [103, 119]], 
                   dtype='float32')

inputs = torch.from_numpy(inputs)
labels = torch.from_numpy(labels)

Creating tensorDataset that permits access to ```inputs``` and ```labels``` as tuples

In [ ]:
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader 

In [ ]:
train_ds = TensorDataset(inputs, labels)
train_ds[:1]

(tensor([[73., 67., 43.]]), tensor([[56., 70.]]))

In [ ]:
bs = 5
train_dl = DataLoader(train_ds, batch_size=bs,shuffle=True)
for x,y in train_dl:
  print(f'x:{x}\ny:{y}')
  break

x:tensor([[ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 87., 134.,  58.],
        [ 69.,  96.,  70.],
        [ 69.,  96.,  70.]])
y:tensor([[119., 133.],
        [ 22.,  37.],
        [119., 133.],
        [103., 119.],
        [103., 119.]])


Model creation 

In [ ]:
model = nn.Linear(3,2) #3 input rows, 2 output rows
print(list(model.parameters())) 

[Parameter containing:
tensor([[ 0.2038, -0.2211, -0.0482],
        [-0.2018,  0.0506, -0.4461]], requires_grad=True), Parameter containing:
tensor([-0.0747,  0.2792], requires_grad=True)]


In [ ]:
import torch.nn.functional as F
loss = F.mse_loss(model(inputs),labels)
optimizer = torch.optim.SGD(model.parameters(), lr=1e-5) 
print(loss)

tensor(13123.0166, grad_fn=<MseLossBackward>)


In [ ]:
#function definitions 

def fit(numEpoch,model,loss,optimizer,train_dl):
  for i in range(numEpoch):
    for z,y in train_dl:
      pred = model(x) #Predictions
      loss = F.mse_loss(pred, y) #Loss
      loss.backward() #Gradient Calculation
      optimizer.step() #update the parameters using grads
      optimizer.zero_grad() #Resetting back to zero

#Progress print
if (i+1) % 10==0: 
  

13123.0166015625